In [1]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from scipy.ndimage import correlate
from tqdm import tqdm as tqdm

%matplotlib inline

In [2]:
savingName = 'fancy_light'

In [3]:
vidObj = cv.VideoCapture("./videos/%s.mp4" % savingName )
fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter('./output/%s.avi' % savingName,fourcc, 30.0, (640,480))

In [4]:
imgArray = []
maskArray = []

In [5]:
# success, img1 = vidObj.read()
# img = cv.resize(img1, dsize=(640,480), interpolation=cv.INTER_LINEAR)

# mask = np.zeros(img.shape[:2],np.uint8)

# bgdModel = np.zeros((1,65),np.float64)
# fgdModel = np.zeros((1,65),np.float64)
# rect = (320,180,200,300)



# cv.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv.GC_INIT_WITH_RECT);
# front_mask = np.where((mask==2)|(mask==0),0,1).astype('uint8')
# fore = img*front_mask[:,:,np.newaxis]

# plt.imshow(cv.cvtColor(fore, cv.COLOR_BGR2RGB)),plt.colorbar(),plt.show();

In [6]:
%%time

success = 1
i=1

success, img1 = vidObj.read()
while success:
    img = cv.resize(img1, dsize=(640,480), interpolation=cv.INTER_LINEAR)
    mask = np.zeros(img.shape[:2],np.uint8)
    
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
    rect = (320,180,200,300)
    
    cv.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv.GC_INIT_WITH_RECT);
    front_mask = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    fore = img*front_mask[:,:,np.newaxis]
    out.write(fore)
    print(i)
    i +=1
    
    imgArray.append(img)
    maskArray.append(mask)
    
    success, img1 = vidObj.read()

vidObj.release()
out.release()
print("Ended")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
Ended
Wall time: 3min 48s


In [7]:
maskArray = np.asarray(maskArray)
imgArray = np.asarray(imgArray)

maskArray.shape

(154, 480, 640)

In [8]:
filt = np.ones([10,3,3])

In [9]:
# 1 in foreground
mask = np.where((maskArray==2)|(maskArray==0),0,1).astype('uint8')

In [10]:
c = correlate(mask,filt,mode = 'nearest')


In [11]:

# if number of pixel in neighbourhood >= 10
maskNew = np.where((c >= 60),1,0).astype('uint8')


In [12]:
newout = cv.VideoWriter('./output/smooth%s.avi' % savingName,fourcc, 30.0, (640,480))

In [13]:
imgArray.shape

(154, 480, 640, 3)

In [14]:
img_old = imgArray
mask_old = maskNew

maskNew = maskNew[:120,:,:]
imgArray = imgArray[:120,:,:,:]


In [15]:
for i in range(imgArray.shape[0]):
    img = imgArray[i,:,:,:]
    front_mask = maskNew[i,:,:]
    fore = img*front_mask[:,:,np.newaxis]
    newout.write(fore)
newout.release()

In [24]:
## Smoothing
blurvideo = cv.VideoWriter('./output/blur%s.avi' % savingName,fourcc, 30.0, (640,480))

In [25]:
for i in tqdm(range(imgArray.shape[0])):
    img = imgArray[i,:,:,:]
    front_mask = maskNew[i,:,:]
    fore = img * front_mask[:,:,np.newaxis]
    back = img - fore    
    imgTemp = np.power(img,3.0)
    maxEle = np.amax(imgTemp)
    gamCorrImg=np.divide(imgTemp,maxEle)
    gamCorrImg=gamCorrImg*255
    gamCorrImg=gamCorrImg.astype(np.uint8)
    K = 32
    x = np.arange(0, K)
    y = np.arange(0, K)
    arr = np.zeros((y.size, x.size))
    cx = K/2 - 1.
    cy = K/2 - 1.
    r = K/2 - 1.
    maska = (x[np.newaxis,:]-cx)**2 + (y[:,np.newaxis]-cy)**2 < r**2
    arr[maska] = 1.
    kern=np.divide(arr,np.sum(arr))
    tImg=np.zeros(np.shape(gamCorrImg))
    tImg[:,:,0]=cv.filter2D(gamCorrImg[:,:,0],-1,kern)
    tImg[:,:,1]=cv.filter2D(gamCorrImg[:,:,1],-1,kern)
    tImg[:,:,2]=cv.filter2D(gamCorrImg[:,:,2],-1,kern)
    tImg=tImg.astype(np.uint8)
    bImg=np.power(tImg,(1/3.0))
    maxE = np.amax(bImg)
    bImg=np.divide(bImg,(1.0*maxE))
    bImg=bImg*255
    bImg=bImg.astype(np.uint8)
    tImg2=np.zeros(np.shape(img))
    tImg2[:,:,0]=cv.filter2D(img[:,:,0],-1,kern)
    tImg2[:,:,1]=cv.filter2D(img[:,:,1],-1,kern)
    tImg2[:,:,2]=cv.filter2D(img[:,:,2],-1,kern)
    bImg2=tImg2.astype(np.uint8)
    bImg3 = np.maximum(bImg,bImg2)
    imgNew = fore + bImg3 * (1 - front_mask[:,:,np.newaxis])
    blurvideo.write(imgNew)
blurvideo.release()   

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120/120 [00:52<00:00,  2.29it/s]


In [18]:
normalVideo = cv.VideoWriter('./output/normal%s.avi' % savingName,fourcc, 30.0, (640,480))

In [19]:
for i in tqdm(range(imgArray.shape[0])):
    img = imgArray[i,:,:,:]
    normalVideo.write(img)
normalVideo.release()   

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120/120 [00:00<00:00, 402.21it/s]


In [26]:
a = 2
a

2